In [1]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime 

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell

#Yahoo!Finance 模組
import yfinance as yf 

import warnings
warnings.filterwarnings('ignore')

In [2]:
plt.rcParams['figure.figsize'] = (10, 7.5)
plt.rcParams['axes.grid'] = False

In [3]:
print(tf.__version__)

2.12.0


In [4]:
tf.random.set_seed(42)
np.random.seed(42)

In [5]:
# https://finance.yahoo.com/most-active
# https://finance.yahoo.com/crypto/
symbol = "BTC-USD"
# Read data
df = yf.download(symbol,period='2y')
df_Indicators= yf.download(symbol,period='2y')
df.head()

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-06-25,34659.105469,35487.246094,31350.884766,31637.779297,31637.779297,40230904226
2021-06-26,31594.664062,32637.587891,30184.501953,32186.277344,32186.277344,38585385521
2021-06-27,32287.523438,34656.128906,32071.757812,34649.644531,34649.644531,35511640894
2021-06-28,34679.121094,35219.890625,33902.074219,34434.335938,34434.335938,33892523752
2021-06-29,34475.558594,36542.109375,34252.484375,35867.777344,35867.777344,37901460044


In [6]:
df_Indicators.shape

(731, 6)

In [7]:
def SMA(close,n):   
    return close.rolling(window=n).mean()

def WMA(close, n):
    weights = np.array(range(1, n+1))
    sum_weights = np.sum(weights)

    res = close.rolling(window=n).apply(lambda x: np.sum(weights*x) / sum_weights, raw=False)
    return res

def EMA(close, n):    
    res=close.ewm(span=n,min_periods=n).mean()
    return res

def RSI(close, n):
    # RSI
    change = close.diff(1)
    df["Gain"] = change.mask(change < 0, 0)
    df["Loss"] = abs(change.mask(change > 0, 0))
    df["AVG_Gain"] = df.Gain.rolling(n).mean()
    df["AVG_Loss"] = df.Loss.rolling(n).mean()
    df["RS"] = df["AVG_Gain"] / df["AVG_Loss"]
    df["RSI"] = 100 - (100 / (1 + df["RS"]))
    return df["RSI"]


def Stoch_RSI(close, n):
    #Stochastic RSI
    RSI=df["RSI"]
    LL_RSI = RSI.rolling(n).min()
    HH_RSI = RSI.rolling(n).max()         
    return (RSI - LL_RSI) / (HH_RSI - LL_RSI)

def BB(close, n):
    MA = pd.Series(close.rolling(n).mean())
    STD = pd.Series(close.rolling(n).std())
    bb1 = MA + 2 * STD
    df["Upper Bollinger Band"] = pd.Series(bb1)
    bb2 = MA - 2 * STD
    df["Lower Bollinger Band"] = pd.Series(bb2)
    return df["Upper Bollinger Band"],df["Lower Bollinger Band"]

def MACD(close):
    df['EMA12'] = close.ewm(span=12, adjust=False).mean() 
    df['EMA26'] = close.ewm(span=26, adjust= False).mean() 
    return df['EMA12'] - df['EMA26'] 


# 指標副程式(資料,天數)
df_Indicators['SMA_7']=SMA(df['Adj Close'],7)
df_Indicators['SMA_14']=SMA(df['Adj Close'],14)

df_Indicators['EMA_7']=EMA(df['Adj Close'],7)
df_Indicators['EMA_14']=EMA(df['Adj Close'],14)

df_Indicators['RSI']=RSI(df['Adj Close'],14)


df_Indicators['Stoch_RSI']=Stoch_RSI(df['Adj Close'],14)

df_Indicators['Upper_BB']=BB(df['Adj Close'],20)[0]
df_Indicators['Lower_BB']=BB(df['Adj Close'],20)[1]

df_Indicators['MACD']=MACD(df['Adj Close'])
df_Indicators= df_Indicators.dropna()
df_Indicators.head()

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD
Date,,,,,,,,,,,,,,,
2021-07-22,32138.873047,32576.400391,31745.298828,32313.105469,32313.105469,19555230518,31400.056641,32272.861886,31718.123505,32111.897636,47.135005,0.668763,35600.983279,30048.046018,-428.245315
2021-07-23,32305.958984,33581.550781,32057.892578,33581.550781,33581.550781,22552046192,31708.629743,32257.400391,32184.091275,32310.990204,48.938437,0.794436,35434.955409,30105.374278,-270.260492
2021-07-24,33593.730469,34490.390625,33424.859375,34292.445312,34292.445312,21664706865,32102.826451,32312.537946,32711.273930,32578.844047,53.631296,1.000000,35219.226293,30221.569800,-86.693783
2021-07-25,34290.292969,35364.925781,33881.835938,35350.187500,35350.187500,20856685287,32610.451730,32391.823661,33371.090696,32952.784607,55.060698,1.000000,35530.362655,30070.851798,142.492547
2021-07-26,35384.031250,40499.675781,35287.312500,37337.535156,37337.535156,51022126212,33541.837891,32690.515625,34362.801431,33543.480221,67.614707,1.000000,36309.885146,29601.563291,478.965494


In [8]:
df_Indicators.shape

(704, 15)

In [9]:
n = len(df)
train_df = df_Indicators[0:int(n*0.7)]
val_df = df_Indicators[int(n*0.7):int(n*0.9)]
test_df = df_Indicators[int(n*0.9):]

train_df.shape, val_df.shape, test_df.shape

((511, 15), (146, 15), (47, 15))

In [10]:
#將值縮限在0~1之間，加速訓練時間
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_df)

train_df[train_df.columns] = scaler.transform(train_df[train_df.columns])
val_df[val_df.columns] = scaler.transform(val_df[val_df.columns])
test_df[test_df.columns] = scaler.transform(test_df[test_df.columns])

In [11]:
train_df.to_csv('data/train.csv')
val_df.to_csv('data/val.csv')
test_df.to_csv('data/test.csv')